In [1]:
# === Train Conditional Diffusion Model on VAE Embeddings with Classifier-Based Guidance ===
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import scanpy as sc
import scipy.sparse as sp
import pandas as pd
import anndata as ad
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import umap
import numpy as np
import math
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score, classification_report

/projects/b1042/GoyalLab/jaekj/python/DL_py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# === Ground truth ===
import os
data_dir = "/projects/b1042/GoyalLab/jaekj/perturb-seq"
z_train = np.load(os.path.join(data_dir, "z_train.npy"))
z_val = np.load(os.path.join(data_dir, "z_val.npy"))
z_test = np.load(os.path.join(data_dir, "z_test.npy")) 

y_train = np.load(os.path.join(data_dir, "y_train.npy")).astype(int)
y_val = np.load(os.path.join(data_dir, "y_val.npy")).astype(int)
y_test = np.load(os.path.join(data_dir, "y_test.npy")).astype(int)

In [3]:
y_test

array([ 5, 19, 68, ...,  6, 16, 63], shape=(8477,))

In [4]:
y_test_torch = torch.from_numpy(y_test)
unique_labels = torch.unique(y_test_torch)
unique_test_labels = [int(v) for v in unique_labels.numpy()]

In [5]:
unique_test_labels

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99]

In [6]:
unique_classes = np.sort(np.unique(y_test))
# Create new ordered arrays
z_test_ordered = []
y_test_ordered = []

# For each class, collect all samples
for class_label in unique_classes:
    # Find all indices where y_test == class_label
    class_indices = np.where(y_test == class_label)[0]
    
    # Extract embeddings and labels for this class
    z_test_ordered.append(z_test[class_indices])
    y_test_ordered.append(y_test[class_indices])
    
    print(f"Class {class_label}: {len(class_indices)} samples")

# Concatenate all classes
z_test_ordered = np.vstack(z_test_ordered)
y_test_ordered = np.concatenate(y_test_ordered)

Class 0: 339 samples
Class 1: 131 samples
Class 2: 68 samples
Class 3: 117 samples
Class 4: 70 samples
Class 5: 68 samples
Class 6: 80 samples
Class 7: 122 samples
Class 8: 31 samples
Class 9: 86 samples
Class 10: 84 samples
Class 11: 63 samples
Class 12: 326 samples
Class 13: 101 samples
Class 14: 79 samples
Class 15: 106 samples
Class 16: 128 samples
Class 17: 75 samples
Class 18: 91 samples
Class 19: 272 samples
Class 20: 55 samples
Class 21: 56 samples
Class 22: 60 samples
Class 23: 60 samples
Class 24: 54 samples
Class 25: 69 samples
Class 26: 116 samples
Class 27: 48 samples
Class 28: 98 samples
Class 29: 106 samples
Class 30: 78 samples
Class 31: 81 samples
Class 32: 46 samples
Class 33: 81 samples
Class 34: 79 samples
Class 35: 89 samples
Class 36: 81 samples
Class 37: 86 samples
Class 38: 60 samples
Class 39: 332 samples
Class 40: 62 samples
Class 41: 76 samples
Class 42: 112 samples
Class 43: 65 samples
Class 44: 67 samples
Class 45: 62 samples
Class 46: 49 samples
Class 47: 

In [7]:
y_test_ordered

array([ 0,  0,  0, ..., 99, 99, 99], shape=(8477,))

In [8]:
import xgboost as xgb

dtrain = xgb.DMatrix(z_train, label=y_train)
dval   = xgb.DMatrix(z_val,   label=y_val)
dtest  = xgb.DMatrix(z_test_ordered)

params = {
    "max_depth": 6,
    "eta": 0.1,
    "objective": "multi:softprob",   # get probabilities
    "num_class": len(np.unique(y_train)),
    "eval_metric": "mlogloss",
    "tree_method": "hist",
}

evallist = [(dval, "eval"), (dtrain, "train")]

bst = xgb.train(
    params,
    dtrain,
    num_boost_round=500,
    evals=evallist,
    early_stopping_rounds=20,
    verbose_eval=True
)

posteriors_xgb = bst.predict(dtest)         # shape [N, C]
y_pred_xgb = posteriors_xgb.argmax(axis=1)

[0]	eval-mlogloss:1.37009	train-mlogloss:1.31523
[1]	eval-mlogloss:1.23441	train-mlogloss:1.17068
[2]	eval-mlogloss:1.13312	train-mlogloss:1.05966
[3]	eval-mlogloss:1.04891	train-mlogloss:0.96745
[4]	eval-mlogloss:0.97775	train-mlogloss:0.88860
[5]	eval-mlogloss:0.91631	train-mlogloss:0.81965
[6]	eval-mlogloss:0.86230	train-mlogloss:0.75831
[7]	eval-mlogloss:0.81462	train-mlogloss:0.70415
[8]	eval-mlogloss:0.77197	train-mlogloss:0.65508
[9]	eval-mlogloss:0.73356	train-mlogloss:0.61064
[10]	eval-mlogloss:0.69919	train-mlogloss:0.57050
[11]	eval-mlogloss:0.66854	train-mlogloss:0.53377
[12]	eval-mlogloss:0.64089	train-mlogloss:0.50028
[13]	eval-mlogloss:0.61592	train-mlogloss:0.46938
[14]	eval-mlogloss:0.59297	train-mlogloss:0.44118
[15]	eval-mlogloss:0.57225	train-mlogloss:0.41507
[16]	eval-mlogloss:0.55303	train-mlogloss:0.39088
[17]	eval-mlogloss:0.53531	train-mlogloss:0.36870
[18]	eval-mlogloss:0.51933	train-mlogloss:0.34807
[19]	eval-mlogloss:0.50475	train-mlogloss:0.32901
[20]	eval-

In [9]:
np.save(f"posteriors_xgb.npy", posteriors_xgb)
np.save(f"y_pred_xgb.npy", y_pred_xgb)

# Save the model
bst.save_model(f"xgb_model.json")

print(f"\n[Info] Saved:")
print(f" - XGBoost model → xgb_model.json")
print(f" - posteriors_xgb.npy")
print(f" - y_pred_xgb.npy")


[Info] Saved:
 - XGBoost model → xgb_model.json
 - posteriors_xgb.npy
 - y_pred_xgb.npy


In [1]:
import xgboost as xgb
print(xgb.__version__)

3.1.1
